In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import io
import zipfile
import re
import os

def fetch_gdelt_gkg_daily(date_str):
    """
    특정 날짜의 GDELT GKG 2.0 데이터 수집 (Bitcoin 관련 영어 기사만)
    
    Parameters:
    -----------
    date_str : str
        수집할 날짜 (YYYYMMDD 형식, 예: '20250901')
    
    Returns:
    --------
    list : 수집된 뉴스 데이터 리스트 (각 항목은 딕셔너리)
    """
    news_data = []  # 수집한 뉴스를 저장할 리스트
    
    # 하루를 24시간으로 나누어 각 시간마다 데이터 수집
    for hour in range(0, 24):
        # 시간 문자열 생성 (예: 00시 -> "000000")
        time_str = f"{hour:02d}0000"
        
        # GDELT GKG 2.0 파일 URL 생성
        # 예: http://data.gdeltproject.org/gdeltv2/20250901000000.gkg.csv.zip
        gkg_url = f"http://data.gdeltproject.org/gdeltv2/{date_str}{time_str}.gkg.csv.zip"
        
        try:
            # HTTP 요청으로 ZIP 파일 다운로드 (10초 타임아웃)
            response = requests.get(gkg_url, timeout=10)
            
            # 요청 성공 시 (상태 코드 200)
            if response.status_code == 200:
                hour_count = 0  # 현재 시간대에서 수집한 기사 수
                
                # ZIP 파일 압축 해제
                with zipfile.ZipFile(io.BytesIO(response.content)) as z:
                    # ZIP 내부의 모든 파일 순회
                    for filename in z.namelist():
                        with z.open(filename) as f:
                            # CSV 파일 읽기
                            # - sep='\t': 탭으로 구분된 데이터
                            # - header=None: 헤더 없음
                            # - dtype=str: 모든 데이터를 문자열로 읽기
                            # - on_bad_lines='skip': 잘못된 행은 건너뛰기
                            # - encoding_errors='ignore': 인코딩 오류 무시
                            df = pd.read_csv(f, sep='\t', header=None, dtype=str, 
                                           on_bad_lines='skip', encoding='utf-8', 
                                           encoding_errors='ignore')
                            
                            # CSV의 각 행(기사) 처리
                            for idx, row in df.iterrows():
                                # 시간당 10개 제한에 도달하면 중단
                                if hour_count >= 10:
                                    break
                                    
                                try:
                                    # GKG 2.0 CSV는 최소 16개 컬럼 필요
                                    if len(row) > 15:
                                        # ===== 1. 기본 정보 추출 =====
                                        # row[4]: DocumentIdentifier (기사 URL)
                                        url = str(row[4]).strip() if len(row) > 4 else ''
                                        
                                        # row[3]: SourceCommonName (출처)
                                        source = str(row[3]).strip() if len(row) > 3 else ''
                                        
                                        # row[7]: V2Themes (주제/테마)
                                        v2_themes = str(row[7]).strip() if len(row) > 7 else ''
                                        
                                        # ===== 2. Bitcoin 키워드 필터링 =====
                                        # URL, 출처, 테마를 합쳐서 Bitcoin 관련 키워드 검색
                                        bitcoin_keywords = ['bitcoin', 'btc', 'cryptocurrency', 'crypto']
                                        text_to_check = f"{url} {source} {v2_themes}".lower()
                                        
                                        # 키워드가 하나도 없으면 건너뛰기
                                        if not any(keyword in text_to_check for keyword in bitcoin_keywords):
                                            continue
                                        
                                        # ===== 3. 영어 기사만 필터링 =====
                                        # 비영어권 도메인을 가진 URL 제외
                                        non_english_domains = ['.cn', '.jp', '.kr', '.ru', '.de', '.fr', '.es', '.it']
                                        if any(domain in url.lower() for domain in non_english_domains):
                                            continue
                                        
                                        # ===== 4. Tone 데이터 추출 =====
                                        # row[15]: Tone (감정 점수)
                                        # 형식: "AvgTone,PositiveScore,NegativeScore,Polarity,ActivityRefDensity,SelfRefDensity,WordCount"
                                        tone_str = str(row[15]).strip()
                                        tone_parts = tone_str.split(',')  # 쉼표로 분리
                                        
                                        # 각 Tone 값 추출 (값이 없으면 0.0)
                                        avg_tone = float(tone_parts[0]) if len(tone_parts) > 0 and tone_parts[0] else 0.0
                                        positive_score = float(tone_parts[1]) if len(tone_parts) > 1 and tone_parts[1] else 0.0
                                        negative_score = float(tone_parts[2]) if len(tone_parts) > 2 and tone_parts[2] else 0.0
                                        polarity = float(tone_parts[3]) if len(tone_parts) > 3 and tone_parts[3] else 0.0
                                        
                                        # ===== 5. 데이터 저장 =====
                                        # 딕셔너리 형태로 뉴스 데이터 추가
                                        news_data.append({
                                            'date': date_str,                    # 날짜
                                            'time': f"{hour:02d}:00",            # 시간
                                            'source': source,                     # 출처
                                            'url': url,                           # URL
                                            'v2_themes': v2_themes,              # 주제
                                            'avg_tone': avg_tone,                # 평균 감정 점수 (-100 ~ +100)
                                            'positive_score': positive_score,    # 긍정 점수 (0~100)
                                            'negative_score': negative_score,    # 부정 점수 (0~100)
                                            'polarity': polarity,                # 극성 (0~100)
                                            'title': 'N/A'                        # 제목 (나중에 추가)
                                        })
                                        
                                        hour_count += 1  # 수집 카운트 증가
                                        
                                except (ValueError, IndexError) as e:
                                    # 데이터 파싱 오류 시 건너뛰기
                                    continue
                
                # 현재 시간대에서 수집한 기사가 있으면 출력
                if hour_count > 0:
                    print(f"✓ {date_str} {hour:02d}:00 완료 (Bitcoin 관련 {hour_count}개)")
        
        except requests.exceptions.RequestException:
            # 네트워크 오류 (파일이 없을 수 있음) - 조용히 건너뛰기
            pass
        except Exception as e:
            # 기타 예외 발생 시 오류 메시지 출력
            print(f"⚠ {date_str} {hour:02d}:00 오류: {str(e)}")
    
    return news_data  # 하루치 수집 데이터 반환


def extract_title_from_url_batch(news_data, batch_size=30):
    """
    배치 단위로 URL에서 제목 추출 (속도 개선을 위해 일부만 처리)
    
    Parameters:
    -----------
    news_data : list
        뉴스 데이터 리스트
    batch_size : int
        제목을 추출할 기사 개수 (기본값: 30개)
    
    Returns:
    --------
    list : 제목이 추가된 뉴스 데이터 리스트
    """
    print(f"  [*] 제목 추출 중 (총 {len(news_data)}개 중 {batch_size}개만)...")
    
    # 지정된 개수만큼만 제목 추출 (너무 많으면 시간이 오래 걸림)
    for idx, item in enumerate(news_data[:batch_size]):
        try:
            url = item['url']
            # URL이 유효한 HTTP 주소인지 확인
            if url and url != 'N/A' and url.startswith('http'):
                # HTTP 요청으로 웹페이지 다운로드 (3초 타임아웃)
                # User-Agent 헤더: 브라우저인 것처럼 위장
                response = requests.get(url, timeout=3, headers={'User-Agent': 'Mozilla/5.0'})
                
                # 요청 성공 시
                if response.status_code == 200:
                    # HTML에서 <title> 태그 찾기 (정규표현식 사용)
                    title_match = re.search(r'<title>(.*?)</title>', response.text, re.IGNORECASE)
                    if title_match:
                        # 제목 추출 및 앞뒤 공백 제거
                        item['title'] = title_match.group(1).strip()
        except Exception:
            # 오류 발생 시 기본값 유지 ('N/A')
            pass
    
    return news_data


def save_daily_csv(news_data, date_str, output_dir='./data/news'):
    """
    하루치 데이터를 CSV 파일로 저장
    
    Parameters:
    -----------
    news_data : list
        저장할 뉴스 데이터 리스트
    date_str : str
        날짜 문자열 (YYYYMMDD)
    output_dir : str
        저장할 디렉토리 경로 (기본값: './data/news')
    """
    # 저장할 데이터가 없으면 종료
    if len(news_data) == 0:
        print(f"  ⚠ {date_str} 저장할 데이터 없음")
        return
    
    # 출력 디렉토리가 없으면 생성
    # exist_ok=True: 이미 존재해도 오류 발생 안 함
    os.makedirs(output_dir, exist_ok=True)
    
    # 리스트를 DataFrame(표 형태)으로 변환
    df = pd.DataFrame(news_data)
    
    # 컬럼 순서 지정
    columns = ['date', 'time', 'title', 'source', 'url', 'v2_themes', 
               'avg_tone', 'positive_score', 'negative_score', 'polarity']
    # 지정된 컬럼만 선택
    df = df[[c for c in columns if c in df.columns]]
    
    # 파일명 생성 (예: bitcoin_news_20250901.csv)
    filename = os.path.join(output_dir, f'bitcoin_news_{date_str}.csv')
    
    # CSV 파일로 저장
    # - index=False: 인덱스 번호 제외
    # - encoding='utf-8-sig': 한글 및 특수문자 지원 (Excel 호환)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    
    # 통계 정보 출력
    avg_tone = df['avg_tone'].astype(float).mean()  # 평균 Tone 점수
    print(f"  ✓ {date_str} 저장 완료: {len(df)}개 기사, 평균 Tone: {avg_tone:.2f}")


def fetch_and_save_daily(start_date, end_date):
    """
    날짜 범위 동안 매일 데이터를 수집하고 저장
    
    Parameters:
    -----------
    start_date : str
        시작 날짜 (YYYYMMDD)
    end_date : str
        종료 날짜 (YYYYMMDD)
    """
    # 문자열을 datetime 객체로 변환
    current_date = datetime.strptime(start_date, '%Y%m%d')
    end = datetime.strptime(end_date, '%Y%m%d')
    
    total_count = 0  # 전체 수집 기사 수
    
    # 시작일부터 종료일까지 하루씩 증가하며 반복
    while current_date <= end:
        # datetime을 문자열로 변환 (YYYYMMDD)
        date_str = current_date.strftime('%Y%m%d')
        
        print(f"\n[{date_str}] 데이터 수집 중...")
        
        # ===== 1. 하루치 데이터 수집 =====
        news_data = fetch_gdelt_gkg_daily(date_str)
        
        # ===== 2. 데이터가 있으면 처리 =====
        if len(news_data) > 0:
            # 제목 추출 (시간이 오래 걸리므로 30개만)
            news_data = extract_title_from_url_batch(news_data, batch_size=30)
            
            # CSV 파일로 저장
            save_daily_csv(news_data, date_str)
            
            # 전체 카운트 누적
            total_count += len(news_data)
        else:
            print(f"  ⚠ {date_str} 데이터 없음")
        
        # ===== 3. 다음 날로 이동 =====
        current_date += timedelta(days=1)
    
    # 전체 완료 메시지
    print(f"\n✅ 전체 완료: 총 {total_count}개 기사 저장")


# ===== 메인 실행 부분 =====
if __name__ == "__main__":
    # 수집할 날짜 범위 설정
    start_date = "20250901"  # 2025년 9월 1일
    end_date = "20250930"    # 2025년 9월 30일
    
    print("GDELT GKG 2.0 Bitcoin 관련 뉴스 수집 (일별 저장)")
    print(f"기간: {start_date} ~ {end_date}")
    print("-" * 60)
    
    # 데이터 수집 및 저장 실행
    fetch_and_save_daily(start_date, end_date)

In [ ]:
import pandas as pd
import os
from datetime import datetime, timedelta

def merge_bitcoin_news_csv(start_date, end_date, input_dir='./data/news', output_file='bitcoin_news_merged.csv'):
    """
    날짜 범위의 Bitcoin 뉴스 CSV 파일들을 하나로 병합
    
    Parameters:
    -----------
    start_date : str
        시작 날짜 (YYYYMMDD 형식, 예: '20250901')
    end_date : str
        종료 날짜 (YYYYMMDD 형식, 예: '20251031')
    input_dir : str
        CSV 파일들이 있는 디렉토리 경로 (기본값: './data/news')
    output_file : str
        병합된 파일의 저장 경로 (기본값: 'bitcoin_news_merged.csv')
    """
    
    # 병합할 데이터를 저장할 리스트
    all_data = []
    
    # 날짜 범위 생성
    current_date = datetime.strptime(start_date, '%Y%m%d')
    end = datetime.strptime(end_date, '%Y%m%d')
    
    # 파일 카운터
    file_count = 0
    success_count = 0
    
    print("=" * 60)
    print("Bitcoin 뉴스 CSV 파일 병합 시작")
    print("=" * 60)
    
    # 시작일부터 종료일까지 하루씩 반복
    while current_date <= end:
        # 날짜를 문자열로 변환 (YYYYMMDD)
        date_str = current_date.strftime('%Y%m%d')
        
        # 파일명 생성 (예: bitcoin_news_20250901.csv)
        filename = os.path.join(input_dir, f'bitcoin_news_{date_str}.csv')
        
        # 파일이 존재하는지 확인
        if os.path.exists(filename):
            try:
                # CSV 파일 읽기
                # - encoding='utf-8-sig': 한글 및 BOM 처리
                df = pd.read_csv(filename, encoding='utf-8-sig')
                
                # 데이터프레임을 리스트에 추가
                all_data.append(df)
                
                # 성공 카운트 증가
                success_count += 1
                
                # 진행 상황 출력
                print(f"✓ {date_str}: {len(df)}개 기사 로드 완료")
                
            except Exception as e:
                # 파일 읽기 실패 시 오류 메시지 출력
                print(f"✗ {date_str}: 오류 - {str(e)}")
        else:
            # 파일이 없으면 경고 메시지
            print(f"⚠ {date_str}: 파일 없음")
        
        # 전체 파일 카운트 증가
        file_count += 1
        
        # 다음 날로 이동
        current_date += timedelta(days=1)
    
    print("=" * 60)
    
    # 병합할 데이터가 있는지 확인
    if len(all_data) == 0:
        print("⚠ 병합할 데이터가 없습니다!")
        return
    
    # 모든 데이터프레임을 하나로 병합
    # - ignore_index=True: 인덱스를 새로 생성
    merged_df = pd.concat(all_data, ignore_index=True)
    
    # 날짜순으로 정렬 (date 컬럼 기준)
    if 'date' in merged_df.columns:
        merged_df = merged_df.sort_values(by=['date', 'time'])
    
    # 병합된 파일 저장
    # - index=False: 인덱스 번호 제외
    # - encoding='utf-8-sig': Excel 호환
    merged_df.to_csv(output_file, index=False, encoding='utf-8-sig')
    
    # 결과 통계 출력
    print(f"\n📊 병합 결과:")
    print(f"  - 처리된 파일: {success_count}/{file_count}개")
    print(f"  - 총 기사 수: {len(merged_df)}개")
    print(f"  - 저장 위치: {output_file}")
    
    # 날짜별 통계
    if 'date' in merged_df.columns:
        print(f"\n📅 날짜별 기사 수:")
        date_counts = merged_df['date'].value_counts().sort_index()
        for date, count in date_counts.items():
            print(f"  {date}: {count}개")
    
    # Tone 통계 (avg_tone이 있는 경우)
    if 'avg_tone' in merged_df.columns:
        avg_tone = merged_df['avg_tone'].astype(float).mean()
        print(f"\n😊 평균 Tone 점수: {avg_tone:.2f}")
    
    print("=" * 60)
    print("✅ 병합 완료!")


# ===== 메인 실행 부분 =====
if __name__ == "__main__":
    # 병합할 날짜 범위 설정
    start_date = "20250901"  # 2025년 9월 1일
    end_date = "20251031"    # 2025년 10월 31일
    
    # 입력/출력 경로 설정
    input_directory = "./data/news"  # CSV 파일들이 있는 폴더
    output_filename = "bitcoin_news_merged.csv"  # 병합된 파일명
    
    # 병합 실행
    merge_bitcoin_news_csv(
        start_date=start_date,
        end_date=end_date,
        input_dir=input_directory,
        output_file=output_filename
    )

In [ ]:
import pandas as pd

def remove_title_column(input_file='bitcoin_news_merged.csv', output_file='bitcoin_news_merged.csv'):
    """
    CSV 파일에서 title 컬럼 제거
    
    Parameters:
    -----------
    input_file : str
        입력 CSV 파일 경로 (기본값: 'bitcoin_news_merged.csv')
    output_file : str
        출력 CSV 파일 경로 (기본값: 'bitcoin_news_merged.csv' - 덮어쓰기)
    """
    
    print("=" * 60)
    print("Title 컬럼 제거 작업 시작")
    print("=" * 60)
    
    try:
        # CSV 파일 읽기
        print(f"\n[*] 파일 로드 중: {input_file}")
        df = pd.read_csv(input_file, encoding='utf-8-sig')
        
        # 로드된 데이터 정보 출력
        print(f"✓ 총 {len(df)}개 행 로드 완료")
        print(f"✓ 기존 컬럼: {list(df.columns)}")
        
        # title 컬럼이 있는지 확인
        if 'title' in df.columns:
            # title 컬럼 제거
            df = df.drop(columns=['title'])
            print(f"\n✓ 'title' 컬럼 제거 완료")
        else:
            print(f"\n⚠ 'title' 컬럼이 존재하지 않습니다")
        
        # 새로운 컬럼 목록 출력
        print(f"✓ 새 컬럼: {list(df.columns)}")
        
        # 파일 저장
        print(f"\n[*] 파일 저장 중: {output_file}")
        df.to_csv(output_file, index=False, encoding='utf-8-sig')
        
        print(f"✓ 저장 완료!")
        print("=" * 60)
        print("✅ 작업 완료!")
        
    except FileNotFoundError:
        print(f"✗ 오류: '{input_file}' 파일을 찾을 수 없습니다")
    except Exception as e:
        print(f"✗ 오류 발생: {str(e)}")


# ===== 메인 실행 부분 =====
if __name__ == "__main__":
    # title 컬럼 제거 실행
    remove_title_column(
        input_file='bitcoin_news_merged.csv',
        output_file='bitcoin_news_no_title.csv'  # 다른 파일로 덮어쓰기
    )